# Data Cleaning
Convert data to a format that the library likes.

In [1]:
import os

import manga109api
import json

In [2]:
manga109_root = "../datasets/Manga109/Manga109_released_2021_12_30"
dataset = manga109api.Parser(manga109_root)

In [29]:
dataset.get_annotation("AisazuNihaIrarenai")["page"][9]["text"]

[{'@id': '00000b18',
  '@xmin': 315,
  '@ymin': 727,
  '@xmax': 359,
  '@ymax': 862,
  '#text': '麻美さんひとりがやけに楽しそう',
  'type': 'text'},
 {'@id': '00000b1b',
  '@xmin': 122,
  '@ymin': 728,
  '@xmax': 160,
  '@ymax': 829,
  '#text': 'なんかヘンなムード',
  'type': 'text'},
 {'@id': '00000b1c',
  '@xmin': 1252,
  '@ymin': 732,
  '@xmax': 1284,
  '@ymax': 813,
  '#text': 'そういえば……',
  'type': 'text'},
 {'@id': '00000b1d',
  '@xmin': 314,
  '@ymin': 98,
  '@xmax': 512,
  '@ymax': 250,
  '#text': 'い…いえ\u3000まだ授業の時間帯と重なっちゃうのが多くて…\nでもちゃんと自分で見つけますから',
  'type': 'text'},
 {'@id': '00000b20',
  '@xmin': 1491,
  '@ymin': 732,
  '@xmax': 1549,
  '@ymax': 849,
  '#text': 'おお？なんかうそっぽいなー',
  'type': 'text'},
 {'@id': '00000b23',
  '@xmin': 654,
  '@ymin': 725,
  '@xmax': 720,
  '@ymax': 815,
  '#text': 'やだなによ今のいい方',
  'type': 'text'},
 {'@id': '00000b28',
  '@xmin': 118,
  '@ymin': 141,
  '@xmax': 159,
  '@ymax': 234,
  '#text': 'やっぱりねまだか',
  'type': 'text'},
 {'@id': '00000b29',
  '@xmin': 481,
  '@ymin': 971,


In [30]:
import shutil
import base64
import numpy as np

os.mkdir("../datasets/Manga109/labelme")
os.mkdir("../datasets/Manga109/labelme/train")
os.mkdir("../datasets/Manga109/labelme/test")
labelme_root = "../datasets/Manga109/labelme/"
for book in dataset.books:
    for page in dataset.get_annotation(book)["page"]:
        # train test split (test size = 0.2)
        if np.random.uniform() > 0.2:
            path = labelme_root + "train/"
        else:
            path = labelme_root + "test/"

        image_path = path + book + str(page["@index"]).zfill(3) + ".jpg"
        shutil.copyfile(manga109_root + "/images/" + book + "/" + str(page["@index"]).zfill(3) + ".jpg", image_path) # wasting space
        encoded = base64.b64encode(open(image_path, "rb").read()) # converting the image into a string for fun
        result = {"imagePath": book + str(page["@index"]).zfill(3) + ".jpg", "imageHeight": page["@height"],
                  "imageWidth": page["@width"], "imageData": str(encoded)[2:-2]}

        #adding shape labels
        shapes = []
        for frame in page["frame"]:
            shapes.append({"label": "frame", "shape_type": "rectangle", "points": [[frame["@xmin"], frame["@ymin"]], [frame["@xmin"], frame["@ymax"]], [frame["@xmax"], frame["@ymin"]], [frame["@xmax"], frame["@ymax"]]]})
        for text in page["text"]:
            shapes.append({"label": "text", "shape_type": "rectangle", "points": [[text["@xmin"], text["@ymin"]], [text["@xmin"], text["@ymax"]], [text["@xmax"], text["@ymin"]], [text["@xmax"], text["@ymax"]]]})
        result["shapes"] = shapes

        # write json
        file = open(path + book + str(page["@index"]).zfill(3) + ".json", "w")
        json.dump(result, file)
        file.close()

# Training Segmentation Model

In [3]:
import pixellib
from pixellib.custom_train import instance_custom_training

vis_img = instance_custom_training()
vis_img.load_dataset("Nature")
vis_img.visualize_sample()

There are 600 listed files in folder train.


Converting labelme annotations to COCO format:   0%|          | 0/600 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'D:\\python_code\\Project-Fugu-Manga-Translator\\ocr\\Nature\\train\\..\\butterfly (1).png'